In [1]:
import os
import pandas as pd
import numpy as np
import math
# 移动均值平滑
def moving_average(data, window_size):
    smoothed_data = []
    for i in range(len(data) - window_size + 1):
        window_data = data[i : i + window_size]
        window_avg = sum(window_data) / window_size
        smoothed_data.append(window_avg)
    return smoothed_data







In [2]:
#加速度
import numpy as np
import math

#速度
def mocap_speed(data): 
    
    # 计算位移（距离差）
    delta_x = np.diff(data['X19']/1000)
    delta_y = np.diff(data['Y19']/1000)
    delta_z = np.diff(data['Z19']/1000)
    # 计算时间差
    delta_t = np.diff(data['Time'])
    # 计算速度
    velocity_x = abs(delta_x / delta_t)
    velocity_y = abs(delta_y / delta_t)
    velocity_z = abs(delta_z / delta_t)
    return velocity_x, velocity_y, velocity_z

#髋关节、膝关节、踝关节角度
def mocap_angle(data):
    #13,15,17号点构成膝关节角度
    knee13_15=data[['X13','Y13','Z13']].values-data[['X15','Y15','Z15']].values
    knee15_17=data[['X17','Y17','Z17']].values-data[['X15','Y15','Z15']].values
    # 计算向量的模
    mod_knee13_15 = np.linalg.norm(knee13_15, axis=1)
    mod_knee15_17 = np.linalg.norm(knee15_17, axis=1)
    # 计算向量的点乘
    dot_knee = (knee13_15 * knee15_17).sum(axis=1)
    # 计算夹角的余弦值
    cos_knee = dot_knee / (mod_knee13_15 * mod_knee15_17)
    
    # 计算夹角的弧度值和角度值
    theta_knee = np.arccos(cos_knee)
    theta_deg_knee = np.rad2deg(theta_knee)
    
    #17,19,21号点构成踝关节角度
    ankle17_19=data[['X17','Y17','Z17']].values-data[['X19','Y19','Z19']].values
    ankle19_21=data[['X21','Y21','Z21']].values-data[['X19','Y19','Z19']].values
    
    # 计算向量的模
    mod_ankle17_19 = np.linalg.norm(ankle17_19, axis=1)
    mod_ankle19_21 = np.linalg.norm(ankle19_21, axis=1)
    # 计算向量的点乘
    dot_ankle = (ankle17_19 * ankle19_21).sum(axis=1)
    # 计算夹角的余弦值
    cos_ankle = dot_ankle / (mod_ankle17_19 * mod_ankle19_21)
    
    # 计算夹角的弧度值和角度值
    theta_ankle = np.arccos(cos_ankle)
    theta_deg_ankle = np.rad2deg(theta_ankle)
    
    return theta_deg_knee,theta_deg_ankle



In [3]:
def statistic(data):
    #数据先平滑
    data1 = moving_average(data, 3)
    len_ = len(data1)
    mean_ = np.mean(data1) #均值
    std_ = np.std(data1) #标准差
    var_ = np.var(data1) #方差
    max_ = max(data1) #最大值
    min_ = min(data1) #最小值
    pk_ = max(abs(max_),abs(min_)) #峰值
    mid_ = np.median(data1) #中位数
    rms_=0
    abs_xbar=0
    r_ = 0
    s = 0
    k = 0
    for i in range(len_):
        rms_ += data1[i] ** 2
        abs_xbar += abs(data1[i])
        r_ += math.sqrt(abs(data1[i]))
        s += (data1[i] - mean_) ** 3
        k += (data1[i] - mean_) ** 4
    rms_ = math.sqrt(rms_ / len_)  # 均方根值
    abs_xbar = abs_xbar / len_# 绝对平均值
    r_ = (r_ / len_) ** 2  # 方根幅值
    s_ = s / ((len_ - 1) * std_ ** 3)  # 偏斜度
    k_ = k / ((len_ - 1) * std_ ** 4)  # 峭度
    return mean_, var_, std_,  max_, min_, pk_, mid_, rms_, abs_xbar, r_, s_, k_

In [11]:

#提取经典统计量：均值、方差、标准差、最大值、最小值、峰值、中位数、均方根、方根幅值、偏度、峰度
def mocap_ki_parameter(file_path):
    # 创建数据框mocap_para，列名（样本，first/second,go/back,task,经典统计量),以左踝为例
    mocap_kinematic = pd.DataFrame(columns=['device','sample', 'times', 'process', 'task', 
                                       'Xspeed_mean','Xspeed_var', 'Xspeed_std', 'Xspeed_max', 'Xspeed_min', 'Xspeed_pk', 'Xspeed_mid','Xspeed_rms','Xspeed_abs_xbar','Xspeed_r','Xspeed_S','Xspeed_K',
                                       'Yspeed_mean','Yspeed_var', 'Yspeed_std', 'Yspeed_max', 'Yspeed_min', 'Yspeed_pk', 'Yspeed_mid','Yspeed_rms','Yspeed_abs_xbar','Yspeed_r','Yspeed_S','Yspeed_K',
                                       'Zspeed_mean','Zspeed_var', 'Zspeed_std', 'Zspeed_max', 'Zspeed_min', 'Zspeed_pk', 'Zspeed_mid','Zspeed_rms','Zspeed_abs_xbar','Zspeed_r','Zspeed_S','Zspeed_K',
                                       'knee_mean','knee_var', 'knee_std', 'knee_max', 'knee_min', 'knee_pk', 'knee_mid','knee_rms','knee_abs_xbar','knee_r','knee_S','knee_K',
                                       'ankle_mean','ankle_var', 'ankle_std', 'ankle_max', 'ankle_min', 'ankle_pk', 'ankle_mid','ankle_rms','ankle_abs_xbar','ankle_r','ankle_S','ankle_K'
                                       ])
    t = 0
    # 读取mocap目录下的first/second
    contents = os.listdir(file_path)
    for item in contents[:]:
        item_path = os.path.join(file_path, item)  # 样本文件夹所在位置
        print('folder:' + item_path)
        content_item = os.listdir(item_path)

        # 读取文件夹go/back/turn
        for process in content_item[:]:
            process_path = os.path.join(item_path, process)
            print('folder:' + process_path)
            process_contents = os.listdir(process_path)

            # 读取文件夹
            for task in process_contents:
                task_path = os.path.join(process_path, task)
                print("folder:" + task_path)
                task_contents = os.listdir(task_path)
                # 读取文件01-23个样本
                for num in task_contents:
                    num_path = os.path.join(task_path, num)
                    print("file:" + num_path)
                    # 读取文件
                    num1 = pd.read_csv(num_path)
                    if len(num1)<1:
                        continue
                    #x的参数
                    speed_x,speed_y,speed_z = mocap_speed(num1)
                    knee,ankle = mocap_angle(num1)
                    
                    
                    xspeed_mean,xspeed_var, xspeed_std, xspeed_max, xspeed_min, xspeed_pk, xspeed_mid,xspeed_rms,xspeed_abs_xbar,xspeed_r,xspeed_s,xspeed_k=statistic(speed_x)
                    yspeed_mean,yspeed_var, yspeed_std, yspeed_max, yspeed_min, yspeed_pk, yspeed_mid,yspeed_rms,yspeed_abs_xbar,yspeed_r,yspeed_s,yspeed_k=statistic(speed_y)
                    zspeed_mean,zspeed_var, zspeed_std, zspeed_max, zspeed_min, zspeed_pk, zspeed_mid,zspeed_rms,zspeed_abs_xbar,zspeed_r,zspeed_s,zspeed_k=statistic(speed_z)

                    knee_mean,knee_var, knee_std, knee_max, knee_min, knee_pk, knee_mid,knee_rms,knee_abs_xbar,knee_r,knee_s,knee_k=statistic(knee)
                     
                    ankle_mean,ankle_var, ankle_std, ankle_max, ankle_min, ankle_pk, ankle_mid,ankle_rms,ankle_abs_xbar,ankle_r,ankle_s,ankle_k=statistic(ankle)
                    
                    
                    
                    mocap_kinematic.loc[t] = ['MOCAP',num[:8], item, process, task,xspeed_mean,xspeed_var, xspeed_std, xspeed_max, xspeed_min, xspeed_pk, xspeed_mid,xspeed_rms,xspeed_abs_xbar,xspeed_r,xspeed_s,xspeed_k,yspeed_mean,yspeed_var, yspeed_std, yspeed_max, yspeed_min, yspeed_pk, yspeed_mid,yspeed_rms,yspeed_abs_xbar,yspeed_r,yspeed_s,yspeed_k,zspeed_mean,zspeed_var, zspeed_std, zspeed_max, zspeed_min, zspeed_pk, zspeed_mid,zspeed_rms,zspeed_abs_xbar,zspeed_r,zspeed_s,zspeed_k, knee_mean,knee_var, knee_std, knee_max, knee_min, knee_pk, knee_mid,knee_rms,knee_abs_xbar,knee_r,knee_s,knee_k,ankle_mean,ankle_var, ankle_std, ankle_max, ankle_min, ankle_pk, ankle_mid,ankle_rms,ankle_abs_xbar,ankle_r,ankle_s,ankle_k]
                    
                    t = t + 1

    mocap_kinematic.to_csv('C:\\Users\\lyy\\Desktop\\Data\\Parameters\\MOCAP_2m\\kinematic_parameters.csv',index = False)





In [12]:
mocap_ki_parameter("C:\\Users\\lyy\\Desktop\\Data\\Processed\\MOCAP_2m")

文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01
文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01\back
文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01\back\task_cup
文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01\back\task_cup\sample01_task_cup.csv
文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01\back\task_cup\sample02_task_cup.csv
文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01\back\task_cup\sample03_task_cup.csv
文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01\back\task_cup\sample04_task_cup.csv
文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01\back\task_cup\sample05_task_cup.csv
文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01\back\task_cup\sample06_task_cup.csv
文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01\back\task_cup\sample07_task_cup.csv
文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01\back\task_cup\sample08_task_cup.csv
文件夹：C:\Users\lyy\Desktop\Data\Processed\MOCAP_2m\time01\back\task_cup\sample0

# kinect提取经典参数

In [15]:
#加速度
import numpy as np
import math


#速度
def kinect_speed(data):
    # 计算位移（距离差）
    delta_x = np.diff(data['X14'])
    delta_y = np.diff(data['Y14'])
    delta_z = np.diff(data['Z14'])
    # 计算时间差
    delta_t = np.diff(data['timestamp2'])
    # 计算速度
    velocity_x = abs(delta_x / delta_t)
    velocity_y = abs(delta_y / delta_t)
    velocity_z = abs(delta_z / delta_t)
    return velocity_x, velocity_y, velocity_z

#膝关节、踝关节角度
def kinect_angle(data):
    #12,13,14号点构成膝关节角度
    knee12_13=data[['X12','Y12','Z12']].values-data[['X13','Y13','Z13']].values
    knee13_14=data[['X14','Y14','Z14']].values-data[['X13','Y13','Z13']].values
    # 计算向量的模
    mod_knee12_13 = np.linalg.norm(knee12_13, axis=1)
    mod_knee13_14 = np.linalg.norm(knee13_14, axis=1)
    # 计算向量的点乘
    dot_knee = (knee12_13 * knee13_14).sum(axis=1)
    # 计算夹角的余弦值
    cos_knee = dot_knee / (mod_knee12_13 * mod_knee13_14)
    
    # 计算夹角的弧度值和角度值
    theta_knee = np.arccos(cos_knee)
    theta_deg_knee = np.rad2deg(theta_knee)
    

    #13,14,15号点构成踝关节角度
    ankle13_14=data[['X13','Y13','Z13']].values-data[['X14','Y14','Z14']].values
    ankle14_15=data[['X15','Y15','Z15']].values-data[['X14','Y14','Z14']].values
    
    # 计算向量的模
    mod_ankle13_14 = np.linalg.norm(ankle13_14, axis=1)
    mod_ankle14_15 = np.linalg.norm(ankle14_15, axis=1)
    # 计算向量的点乘
    dot_ankle = (ankle13_14 * ankle14_15).sum(axis=1)
    # 计算夹角的余弦值
    cos_ankle = dot_ankle / (mod_ankle13_14 * mod_ankle14_15)
    
    # 计算夹角的弧度值和角度值
    theta_ankle = np.arccos(cos_ankle)
    theta_deg_ankle = np.rad2deg(theta_ankle)
    
    return theta_deg_knee,theta_deg_ankle


In [22]:

#提取经典统计量：均值、方差、标准差、最大值、最小值、峰值、中位数、均方根、方根幅值、偏度、峰度
def kinect_ki_parameter(file_path):
    # 创建数据框mocap_para，列名（样本，first/second,go/back,task,经典统计量),以左踝为例
    kinect_kinematic = pd.DataFrame(columns=['device','sample', 'times', 'process', 'task', 
                                       'Xspeed_mean','Xspeed_var', 'Xspeed_std', 'Xspeed_max', 'Xspeed_min', 'Xspeed_pk', 'Xspeed_mid','Xspeed_rms','Xspeed_abs_xbar','Xspeed_r','Xspeed_S','Xspeed_K',
                                       'Yspeed_mean','Yspeed_var', 'Yspeed_std', 'Yspeed_max', 'Yspeed_min', 'Yspeed_pk', 'Yspeed_mid','Yspeed_rms','Yspeed_abs_xbar','Yspeed_r','Yspeed_S','Yspeed_K',
                                       'Zspeed_mean','Zspeed_var', 'Zspeed_std', 'Zspeed_max', 'Zspeed_min', 'Zspeed_pk', 'Zspeed_mid','Zspeed_rms','Zspeed_abs_xbar','Zspeed_r','Zspeed_S','Zspeed_K',
                                       'knee_mean','knee_var', 'knee_std', 'knee_max', 'knee_min', 'knee_pk', 'knee_mid','knee_rms','knee_abs_xbar','knee_r','knee_S','knee_K',
                                       'ankle_mean','ankle_var', 'ankle_std', 'ankle_max', 'ankle_min', 'ankle_pk', 'ankle_mid','ankle_rms','ankle_abs_xbar','ankle_r','ankle_S','ankle_K'
                                       ])
    t = 0
    # 读取kinect目录下的first/second
    contents = os.listdir(file_path)
    for item in contents[:]:
        item_path = os.path.join(file_path, item)  # 样本文件夹所在位置
        print('folder:' + item_path)
        content_item = os.listdir(item_path)

        # 读取文件夹go/back/turn
        for process in content_item[:]:
            process_path = os.path.join(item_path, process)
            print('folder:' + process_path)
            process_contents = os.listdir(process_path)

            # 读取文件夹
            for task in process_contents:
                task_path = os.path.join(process_path, task)
                print("folder:" + task_path)
                task_contents = os.listdir(task_path)
                # 读取文件01-23个样本
                for num in task_contents:
                    num_path = os.path.join(task_path, num)
                    print("file" + num_path)
                    # 读取文件
                    num1 = pd.read_csv(num_path)
                    if len(num1)<1:
                        continue

                    #x的参数

                    speed_x,speed_y,speed_z = kinect_speed(num1)  
                    knee,ankle=kinect_angle(num1)
                    
                    
                    xspeed_mean,xspeed_var, xspeed_std, xspeed_max, xspeed_min, xspeed_pk, xspeed_mid,xspeed_rms,xspeed_abs_xbar,xspeed_r,xspeed_s,xspeed_k=statistic(speed_x)
                    yspeed_mean,yspeed_var, yspeed_std, yspeed_max, yspeed_min, yspeed_pk, yspeed_mid,yspeed_rms,yspeed_abs_xbar,yspeed_r,yspeed_s,yspeed_k=statistic(speed_y)
                    zspeed_mean,zspeed_var, zspeed_std, zspeed_max, zspeed_min, zspeed_pk, zspeed_mid,zspeed_rms,zspeed_abs_xbar,zspeed_r,zspeed_s,zspeed_k=statistic(speed_z)
                    
                    knee_mean,knee_var, knee_std, knee_max, knee_min, knee_pk, knee_mid,knee_rms,knee_abs_xbar,knee_r,knee_s,knee_k=statistic(knee)
                     
                    ankle_mean,ankle_var, ankle_std, ankle_max, ankle_min, ankle_pk, ankle_mid,ankle_rms,ankle_abs_xbar,ankle_r,ankle_s,ankle_k=statistic(ankle)
                    
                    kinect_kinematic.loc[t] = ['Kinect',num[:8], item, process, task,xspeed_mean,xspeed_var, xspeed_std, xspeed_max, xspeed_min, xspeed_pk, xspeed_mid,xspeed_rms,xspeed_abs_xbar,xspeed_r,xspeed_s,xspeed_k,yspeed_mean,yspeed_var, yspeed_std, yspeed_max, yspeed_min, yspeed_pk, yspeed_mid,yspeed_rms,yspeed_abs_xbar,yspeed_r,yspeed_s,yspeed_k,zspeed_mean,zspeed_var, zspeed_std, zspeed_max, zspeed_min, zspeed_pk, zspeed_mid,zspeed_rms,zspeed_abs_xbar,zspeed_r,zspeed_s,zspeed_k,knee_mean,knee_var, knee_std, knee_max, knee_min, knee_pk, knee_mid,knee_rms,knee_abs_xbar,knee_r,knee_s,knee_k,ankle_mean,ankle_var, ankle_std, ankle_max, ankle_min, ankle_pk, ankle_mid,ankle_rms,ankle_abs_xbar,ankle_r,ankle_s,ankle_k]
                    t = t + 1

    kinect_kinematic.to_csv('C:\\Users\\lyy\\Desktop\\Data\\Parameters\\Kinect_2m\\kinematic_parameters.csv',index = False)





In [23]:
kinect_ki_parameter("C:\\Users\\lyy\\Desktop\\Data\\Processed\\Kinect_2m")

folder:C:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01
folder:C:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01\back
folder:C:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01\back\task_cup
fileC:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01\back\task_cup\sample01_task_cup.csv
fileC:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01\back\task_cup\sample02_task_cup.csv
fileC:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01\back\task_cup\sample03_task_cup.csv
fileC:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01\back\task_cup\sample04_task_cup.csv
fileC:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01\back\task_cup\sample05_task_cup.csv
fileC:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01\back\task_cup\sample06_task_cup.csv
fileC:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01\back\task_cup\sample07_task_cup.csv
fileC:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01\back\task_cup\sample08_task_cup.csv
fileC:\Users\lyy\Desktop\Data\Processed\Kinect_2m\time01\

## imuA0提取经典统计量

In [26]:

#提取经典统计量：均值、方差、标准差、最大值、最小值、峰值、中位数、均方根、方根幅值、偏度、峰度
def imu_ki_parameter(file_path):
    # 创建数据框mocap_para，列名（样本，first/second,go/back,task,经典统计量),以左踝为例
    imu_kinematic = pd.DataFrame(columns=['device','sample', 'times', 'process', 'task', 
                                       'Xacc_mean','Xacc_var', 'Xacc_std', 'Xacc_max', 'Xacc_min', 'Xacc_pk', 'Xacc_mid','Xacc_rms','Xacc_abs_xbar','Xacc_r','Xacc_S','Xacc_K',
                                       'Yacc_mean','Yacc_var', 'Yacc_std', 'Yacc_max', 'Yacc_min', 'Yacc_pk', 'Yacc_mid','Yacc_rms','Yacc_abs_xbar','Yacc_r','Yacc_S','Yacc_K',
                                       'Zacc_mean','Zacc_var', 'Zacc_std', 'Zacc_max', 'Zacc_min', 'Zacc_pk', 'Zacc_mid','Zacc_rms','Zacc_abs_xbar','Zacc_r','Zacc_S','Zacc_K',
                                       'Xanv_mean','Xanv_var', 'Xanv_std', 'Xanv_max', 'Xanv_min', 'Xanv_pk', 'Xanv_mid','Xanv_rms','Xanv_abs_xbar','Xanv_r','Xanv_S','Xanv_K',
                                       'Yanv_mean','Yanv_var', 'Yanv_std', 'Yanv_max','Yanv_min', 'Yanv_pk', 'Yanv_mid', 'Yanv_rms','Yanv_abs_xbar','Yanv_r','Yanv_S','Yanv_K',
                                       'Zanv_mean','Zanv_var', 'Zanv_std', 'Zanv_max', 'Zanv_min', 'Zanv_pk', 'Zanv_mid','Zanv_rms','Zanv_abs_xbar','Zanv_r','Zanv_S','Zanv_K',
                                       'Xangle_mean','Xangle_var', 'Xangle_std', 'Xangle_max', 'Xangle_min', 'Xangle_pk', 'Xangle_mid','Xangle_rms','Xangle_abs_xbar','Xangle_r','Xangle_S','Xangle_K',
                                       'Yangle_mean','Yangle_var', 'Yangle_std', 'Yangle_max', 'Yangle_min', 'Yangle_pk', 'Yangle_mid','Yangle_rms','Yangle_abs_xbar','Yangle_r','Yangle_S','Yangle_K',
                                       'Zangle_mean','Zangle_var', 'Zangle_std', 'Zangle_max', 'Zangle_min', 'Zangle_pk', 'Zangle_mid','Zangle_rms','Zangle_abs_xbar','Zangle_r','Zangle_S','Zangle_K'
                                       ])
    t = 0
    # 读取mocap目录下的first/second
    contents = os.listdir(file_path)
    for item in contents[:]:
        item_path = os.path.join(file_path, item)  # 样本文件夹所在位置
        print('folder:' + item_path)
        content_item = os.listdir(item_path)

        # 读取文件夹go/back/turn
        for process in content_item[:]:
            process_path = os.path.join(item_path, process)
            print('folder:' + process_path)
            process_contents = os.listdir(process_path)

            # 读取文件夹
            for task in process_contents:
                task_path = os.path.join(process_path, task)
                print("folder:" + task_path)
                task_contents = os.listdir(task_path)
                # 读取文件01-23个样本
                for num in task_contents:
                    num_path = os.path.join(task_path, num)
                    print("file:" + num_path)
                    # 读取文件
                    num1 = pd.read_csv(num_path)
                    if len(num1)<1:
                        continue
                    
                    xacc_mean,xacc_var, xacc_std, xacc_max, xacc_min, xacc_pk, xacc_mid,xacc_rms,xacc_abs_xbar,xacc_r,xacc_s,xacc_k= statistic(abs(num1['加速度X(g)']*9.8))          #X轴是左右        
                    yacc_mean,yacc_var, yacc_std, yacc_max, yacc_min, yacc_pk, yacc_mid,yacc_rms,yacc_abs_xbar,yacc_r,yacc_s,yacc_k=statistic(abs(num1['加速度Y(g)']*9.8))   #y轴是上下
                    zacc_mean,zacc_var, zacc_std, zacc_max, zacc_min, zacc_pk, zacc_mid,zacc_rms,zacc_abs_xbar,zacc_r,zacc_s,zacc_k=statistic(abs(num1['加速度Z(g)']*9.8))    #z轴是前后
                    
                    
                    xangle_mean,xangle_var, xangle_std, xangle_max, xangle_min, xangle_pk, xangle_mid,xangle_rms,xangle_abs_xbar,xangle_r,xangle_s,xangle_k=statistic(abs(num1['角度X(°)']))
                    yangle_mean,yangle_var, yangle_std, yangle_max, yangle_min, yangle_pk, yangle_mid,yangle_rms,yangle_abs_xbar,yangle_r,yangle_s,yangle_k=statistic(abs(num1['角度Y(°)']))
                    zangle_mean,zangle_var, zangle_std, zangle_max, zangle_min, zangle_pk, zangle_mid,zangle_rms,zangle_abs_xbar,zangle_r,zangle_s,zangle_k=statistic(abs(num1['角度Z(°)']))
                    
                    
                    xanv_mean,xanv_var, xanv_std, xanv_max, xanv_min, xanv_pk, xanv_mid,xanv_rms,xanv_abs_xbar,xanv_r,xanv_s,xanv_k=statistic(abs(num1['角速度X(°/s)']))
                    yanv_mean,yanv_var, yanv_std, yanv_max,yanv_min, yanv_pk, yanv_mid, yanv_rms,yanv_abs_xbar,yanv_r,yanv_s,yanv_k=statistic(abs(num1['角速度Y(°/s)']))
                    zanv_mean,zanv_var, zanv_std, zanv_max, zanv_min,zanv_pk, zanv_mid,zanv_rms,zanv_abs_xbar,zanv_r,zanv_s,zanv_k=statistic(abs(num1['角速度Z(°/s)']))

                    
                    imu_kinematic.loc[t] = ['IMU',num[:8], item, process, task,xacc_mean,xacc_var, xacc_std, xacc_max, xacc_min, xacc_pk, xacc_mid,xacc_rms,xacc_abs_xbar,xacc_r,xacc_s,xacc_k,yacc_mean,yacc_var, yacc_std, yacc_max, yacc_min, yacc_pk, yacc_mid,yacc_rms,yacc_abs_xbar,yacc_r,yacc_s,yacc_k, zacc_mean,zacc_var, zacc_std, zacc_max, zacc_min, zacc_pk, zacc_mid,zacc_rms,zacc_abs_xbar,zacc_r,zacc_s,zacc_k,xanv_mean,xanv_var, xanv_std, xanv_max, xanv_min, xanv_pk, xanv_mid,xanv_rms,xanv_abs_xbar,xanv_r,xanv_s,xanv_k,yanv_mean,yanv_var, yanv_std, yanv_max,yanv_min, yanv_pk, yanv_mid, yanv_rms,yanv_abs_xbar,yanv_r,yanv_s,yanv_k,zanv_mean,zanv_var, zanv_std, zanv_max, zanv_min,zanv_pk, zanv_mid,zanv_rms,zanv_abs_xbar,zanv_r,zanv_s,zanv_k,xangle_mean,xangle_var, xangle_std, xangle_max, xangle_min, xangle_pk, xangle_mid,xangle_rms,xangle_abs_xbar,xangle_r,xangle_s,xangle_k,yangle_mean,yangle_var, yangle_std, yangle_max, yangle_min, yangle_pk, yangle_mid,yangle_rms,yangle_abs_xbar,yangle_r,yangle_s,yangle_k,zangle_mean,zangle_var, zangle_std, zangle_max, zangle_min, zangle_pk, zangle_mid,zangle_rms,zangle_abs_xbar,zangle_r,zangle_s,zangle_k]
                    t = t + 1

    imu_kinematic.to_csv('C:\\Users\\lyy\\Desktop\\Data\\Parameters\\IMU_5m\\kinematic_parameters.csv',index = False)
    #imu_kinematic.to_csv('C:\\Users\\lyy\\Desktop\\Data\\Parameters\\IMU_2m\\kinematic_parameters.csv',index = False)






In [27]:
imu_ki_parameter("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_5m")
#imu_ki_parameter("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_2m")

folder:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01
folder:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01\back
folder:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01\back\task_cup
file:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01\back\task_cup\sample01_task_cup.csv
file:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01\back\task_cup\sample02_task_cup.csv
file:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01\back\task_cup\sample03_task_cup.csv
file:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01\back\task_cup\sample04_task_cup.csv
file:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01\back\task_cup\sample05_task_cup.csv
file:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01\back\task_cup\sample06_task_cup.csv
file:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01\back\task_cup\sample07_task_cup.csv
file:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01\back\task_cup\sample08_task_cup.csv
file:C:\Users\lyy\Desktop\Data\Processed\IMU_5m\time01\back\task_cup\sample09_task